In [1]:
import numpy as np

import openmm
from openmm import Platform, MonteCarloBarostat
import openmm.app as app
import openmm.unit as unit
from openmm.app import Simulation
from openmm import LocalEnergyMinimizer

import mdtraj as md

from utils import _get_openMM_forces as get_forces
from utils import compute_hessian_force_fd_block_serial, compute_hessian_force_fd_block_parallel

## With dimer system

In [2]:
def create_harmonic_dimer(k=300.0, r0=0.1):
    system = openmm.System()
    mass = 12.0 * unit.amu
    system.addParticle(mass)
    system.addParticle(mass)

    # Add custom harmonic bond force
    force = openmm.HarmonicBondForce()
    force.addBond(0, 1, r0 * unit.nanometer, k * unit.kilojoule_per_mole / unit.nanometer**2)
    system.addForce(force)

    # Initial positions: atoms along x-axis at equilibrium distance
    positions = unit.Quantity(
        np.array([[0, 0, 0], [r0, 0, 0]]),
        unit.nanometer
    )
    return system, positions

In [3]:
system, position = create_harmonic_dimer()

In [4]:
def analytical_dimer_hessian(k):
    H = np.zeros((6, 6))
    H[0, 0] = H[3, 3] = k   # x1-x1 and x2-x2
    H[0, 3] = H[3, 0] = -k  # x1-x2 and x2-x1
    return H

def run_numerical_hessian(system, positions, hessian_func, epsilon=1e-4):
    # Use your provided Hessian function (serial or parallel version)
    hessian = hessian_func(system, positions, atom_indices=[0, 1], epsilon=epsilon, platform_name='CPU')
    return hessian

In [5]:
H_analytic = analytical_dimer_hessian(300.0)
print("Analytical Hessian (nonzero part):")
print(H_analytic[:4, :4])  # just x components

Analytical Hessian (nonzero part):
[[ 300.    0.    0. -300.]
 [   0.    0.    0.    0.]
 [   0.    0.    0.    0.]
 [-300.    0.    0.  300.]]


In [6]:
H_numeric = run_numerical_hessian(
    system, position, compute_hessian_force_fd_block_parallel, epsilon=1e-9
)

print("\nNumerical Hessian (block):")
print(H_numeric)


Numerical Hessian (block):
[[ 3.00000000e+02 -2.08166817e-06 -2.08166817e-06 -3.00000000e+02
  -2.08166817e-06 -2.08166817e-06]
 [-2.08166817e-06  4.16333634e-14  0.00000000e+00  2.08166817e-06
  -4.16333634e-14  0.00000000e+00]
 [-2.08166817e-06  0.00000000e+00  4.16333634e-14  2.08166817e-06
   0.00000000e+00 -4.16333634e-14]
 [-3.00000000e+02  2.08166817e-06  2.08166817e-06  3.00000000e+02
   2.08166817e-06  2.08166817e-06]
 [-2.08166817e-06 -4.16333634e-14  0.00000000e+00  2.08166817e-06
   4.16333634e-14  0.00000000e+00]
 [-2.08166817e-06  0.00000000e+00 -4.16333634e-14  2.08166817e-06
   0.00000000e+00  4.16333634e-14]]


In [7]:
# Compare
diff = np.linalg.norm(H_numeric[:6, :6] - H_analytic)
rel_diff = diff / np.linalg.norm(H_analytic)
print(f"\nAbsolute difference: {diff:.3e}")
print(f"Relative difference: {rel_diff:.3e}")


Absolute difference: 8.333e-06
Relative difference: 1.389e-08


In [8]:
def test_hessian_eigenvalues(system, positions, hessian_func, **hessian_kwargs):
    """
    Minimize the system, compute the Hessian, and check eigenvalue structure.
    """
    integrator = openmm.LangevinIntegrator(300 * unit.kelvin, 1/unit.picosecond, 2 * unit.femtoseconds)
    platform = openmm.Platform.getPlatformByName('CPU')
    simulation = Simulation(app.Topology(), system, integrator, platform)
    simulation.context.setPositions(positions)

    # Minimize
    LocalEnergyMinimizer.minimize(simulation.context, tolerance=1)
    state = simulation.context.getState(getPositions=True)
    minimized_positions = state.getPositions()

    state_F = simulation.context.getState(getForces=True)
    max_force = np.max(np.abs(state_F.getForces(asNumpy=True)))
    print("Max force after minimization:", max_force)

    state_e = simulation.context.getState(getEnergy=True, getVelocities=False, getForces=False, getPositions=False)
    energy = state_e.getPotentialEnergy()
    print("OpenMM Pot E (minimized): ", energy)

    # Compute Hessian
    hessian = hessian_func(system, minimized_positions, **hessian_kwargs)

    # Compute eigenvalues
    eigvals = np.linalg.eigvalsh(hessian)
    num_negative = np.sum(eigvals < -1e-6)
    num_near_zero = np.sum(np.abs(eigvals) < 1e-6)

    print(f"Eigenvalues: {eigvals}")
    print(f"Eigenvalues (min, max): {eigvals.min():.3e}, {eigvals.max():.3e}")
    print(f"Negative eigenvalues: {num_negative}")
    print(f"Near-zero eigenvalues (expected ~6 for rigid body modes): {num_near_zero}")

    del simulation

In [9]:
test_hessian_eigenvalues(system, position, hessian_func=compute_hessian_force_fd_block_serial, epsilon=1e-9, atom_indices=None)

Max force after minimization: 0.0
OpenMM Pot E (minimized):  0.0 kJ/mol
Eigenvalues: [-5.19664871e-14  6.67551315e-15  6.62052256e-14  8.32667268e-14
  1.00411450e-13  6.00000000e+02]
Eigenvalues (min, max): -5.197e-14, 6.000e+02
Negative eigenvalues: 0
Near-zero eigenvalues (expected ~6 for rigid body modes): 5


## with ADP system

In [34]:
pdb = app.PDBFile("/Users/arminsh/Documents/GADES/GADES/equilibrated.pdb")
traj = md.load("/Users/arminsh/Documents/GADES/GADES/equilibrated.pdb")
adp_atom_indices = np.array([atom.index for atom in traj.topology.atoms if atom.residue.name != 'HOH']) # what atoms do you want to pick

forcefield = app.ForceField("amber14/protein.ff14SB.xml", 
                        "amber14/tip3p.xml")
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.PME, constraints=None)

integrator = openmm.LangevinIntegrator(300 * unit.kelvin, 1 / unit.picosecond, 2 * unit.femtoseconds)
platform = Platform.getPlatformByName("OpenCL")

simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

state = simulation.context.getState(getEnergy=True, getVelocities=False, getForces=False, getPositions=False)
energy = state.getPotentialEnergy()
print("OpenMM Pot E: ", energy)

OpenMM Pot E:  -12061.050880030336 kJ/mol


In [35]:
def test_force_prediction(system, positions, hessian, atom_indices, epsilon=1e-4, platform_name='CPU'):
    """
    Test if Hessian predicts force changes under random displacement.
    """
    n_atoms = len(positions)
    positions_array = np.array(positions.value_in_unit(unit.nanometer))

    if atom_indices is None:
        atom_indices = np.arange(0, n_atoms)

    coord_indices = []
    for idx in atom_indices:
        coord_indices.extend([3 * idx, 3 * idx + 1, 3 * idx + 2])

    integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
    platform = openmm.Platform.getPlatformByName(platform_name)
    context = openmm.Context(system, integrator, platform)

    positions_nm = positions_array * unit.nanometer
    context.setPositions(positions_nm)

    f0 = get_forces(context, positions_nm)[coord_indices]

    # Random displacement
    delta_x = np.random.randn(len(coord_indices)) * epsilon
    perturbed_positions = positions_array.flatten()
    perturbed_positions[coord_indices] += delta_x
    perturbed_positions = perturbed_positions.reshape((-1, 3)) * unit.nanometer

    f_perturbed = get_forces(context, perturbed_positions)[coord_indices]
    f_predicted = f0 + hessian @ delta_x

    error = np.linalg.norm(f_perturbed - f_predicted) / np.linalg.norm(f_perturbed)
    print(f"Relative force prediction error: {error:.3e}")

    del context
    del integrator

In [36]:
hessian_block = compute_hessian_force_fd_block_serial(system, pdb.positions, atom_indices=None, epsilon=1e-4)
test_force_prediction(system, pdb.positions, hessian_block, atom_indices=None, epsilon=1e-4)

Relative force prediction error: 7.102e-05


In [37]:
def test_hessian_eigenvalues(system, positions, hessian_func, **hessian_kwargs):
    """
    Minimize the system, compute the Hessian, and check eigenvalue structure.
    """
    integrator = openmm.LangevinIntegrator(300 * unit.kelvin, 1/unit.picosecond, 2 * unit.femtoseconds)
    platform = openmm.Platform.getPlatformByName('CPU')
    simulation = Simulation(app.Topology(), system, integrator, platform)
    simulation.context.setPositions(positions)

    # Minimize
    LocalEnergyMinimizer.minimize(simulation.context, tolerance=1e-6)
    state = simulation.context.getState(getPositions=True, getForces=True, getEnergy=True)
    minimized_positions = state.getPositions()

    max_force = np.max(np.abs(state.getForces(asNumpy=True)))
    print("Max force after minimization:", max_force)

    energy = state.getPotentialEnergy()
    print("OpenMM Pot E (minimized): ", energy)

    # Compute Hessian
    hessian = hessian_func(system, minimized_positions, **hessian_kwargs)

    # Compute eigenvalues
    eigvals = np.linalg.eigvalsh(hessian)
    num_negative = np.sum(eigvals < -1e-6)
    num_near_zero = np.sum(np.abs(eigvals) < 1e-6)

    print(f"Eigenvalues (min, max): {eigvals.min():.3e}, {eigvals.max():.3e}")
    print(f"Negative eigenvalues: {num_negative}")
    print(f"Near-zero eigenvalues (expected ~6 for rigid body modes): {num_near_zero}")

    del simulation

In [38]:
test_hessian_eigenvalues(system, pdb.positions, hessian_func=compute_hessian_force_fd_block_serial, epsilon=1e-9, atom_indices=None)

Max force after minimization: 2061.4990234375
OpenMM Pot E (minimized):  -20344.527294915457 kJ/mol
Eigenvalues (min, max): -1.329e+08, 1.376e+08
Negative eigenvalues: 1850
Near-zero eigenvalues (expected ~6 for rigid body modes): 0
